## Importando as bases com os tratamentos da **Etapa** anterior

In [1]:
import pandas as pd
arquivo = '/content/drive/MyDrive/titanic_competition/titanic_train3.csv'
arquivo1 = '/content/drive/MyDrive/titanic_competition/titanic_test3.csv'

treino = pd.read_csv(arquivo)
teste = pd.read_csv(arquivo1)

In [2]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          891 non-null    float64
 4   Fare         891 non-null    float64
 5   male_check   891 non-null    int64  
 6   Familiares   891 non-null    int64  
 7   Embarked     891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [3]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Age          418 non-null    float64
 3   Fare         418 non-null    float64
 4   male_check   418 non-null    int64  
 5   Familiares   418 non-null    int64  
 6   Embarked     418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


### Será utilizado modelos diferentes dos anteriores para fazer a previsão dos dados, para avaliar o desempenho dos mesmos em relação a base já tratada; Manteremos o modelo Regressão Logística, pois está sendo o melhor até o momento.
- Logistic Regression:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html <br>
- Random Forest:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html <br>
- Support Vector Machines:<br>
https://scikit-learn.org/stable/modules/svm.html <br>

E para separação dos dados em **treino** e **validação**:
- train_test_split:<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [4]:
# Bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [5]:
# Separando os dados em X e y
X = treino.drop(['PassengerId', 'Survived'], axis=1)
y = treino.Survived

In [24]:
# Separando em treino e validação
RANDOM_STATE = 1

Xtrain, Xval, ytrain, yval = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [25]:
# Criando os classificadores
clf_log = LogisticRegression(random_state=RANDOM_STATE)
clf_rf = RandomForestClassifier(random_state=RANDOM_STATE)
clf_svm = SVC()

In [26]:
# Ajustando os dados aos modelos
clf_log = clf_log.fit(Xtrain, ytrain)
clf_rf = clf_rf.fit(Xtrain, ytrain)
clf_svm = clf_svm.fit(Xtrain, ytrain)

In [27]:
# Fazendo a previsão dos dados
y_pred_log = clf_log.predict(Xval)
y_pred_rf = clf_rf.predict(Xval)
y_pred_svm = clf_svm.predict(Xval)

Novamente avaliaremos os modelos usando a métrica:
- Acurácia (**método de avaliação usado na competição**):<br>
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [28]:
# Importando a métrica e avaliando os modelos
from sklearn.metrics import accuracy_score
resultado_log = accuracy_score(yval, y_pred_log)
resultado_rf = accuracy_score(yval, y_pred_rf)
resultado_svm = accuracy_score(yval, y_pred_svm)

In [29]:
# Visualizando os resultados
print(f'Regressão Logística: {resultado_log}')
print(f'Random Forest: {resultado_rf}')
print(f'SVM: {resultado_svm}')

Regressão Logística: 0.823728813559322
Random Forest: 0.7898305084745763
SVM: 0.8338983050847457


### Utilizaremos o modelo com a melhor acurácia para fazer a previsão na base de teste

In [30]:
# Verificando os dados de treino
Xtrain.head(3)

,Pclass,Age,Fare,male_check,Familiares,Embarked
6,1,1.814076,0.395814,1,0,0
718,3,-0.052101,-0.336334,1,0,2
685,2,-0.327127,0.188763,1,3,1


In [31]:
# Comparando com os dados de teste
teste.head(3)

,PassengerId,Pclass,Age,Fare,male_check,Familiares,Embarked
0,892,3,0.372316,-0.497079,1,0,2
1,893,3,1.326106,-0.511942,0,1,0
2,894,2,2.470655,-0.463770,1,0,2


In [32]:
# Dropando PassengerId para fazer a previsão
Xtest = teste.drop('PassengerId', axis=1)

In [34]:
# Fazendo a previsão
y_pred = clf_svm.predict(Xtest)

In [36]:
# Criando a coluna alvo a partir da previsão
teste['Survived'] = y_pred

In [37]:
# Separando a base do Kaggle
base_kaggle = teste[['PassengerId', 'Survived']]

In [38]:
# Exportando a base
base_kaggle.to_csv('resultado_e4.csv', index=False)